## ①padding_mask

该篇文章的主要motivation是笔者在进行crosss-attention实战时考虑到的，即当Q!=K=V的时候该如何输入相应的padding_mask。
要考虑上面的问题，我们往前推，self-attention的时候mask相关内容是什么呢，再往前推，embedding的时候mask是考虑什么呢？
那么再从头开始往后推，哪些算子需要输入时提供mask相关内容呢？
笔者略加思考，觉得这部分还是非常重要的，因为若batch中长度分布极度不平衡，这势必会很大程度上影响模型的性能。
下面分别从实践和理论两方面进行相关分析。

### 实践分析

问题的来源：
    batch中的各个句子长度一般并不相同；
本质：
    代码需要并行化的处理多个batch的数据而<pad>位置的信息只会产生噪声（未进行相关实验研究，仅猜测）
那么在进行后续的处理时，我们就需要进行对应位置的遮掩（mask）

#### （1）嵌入时

In [2]:
from transformers import BertTokenizer, RobertaTokenizer
from transformers import BertConfig, BertForPreTraining, RobertaForMaskedLM, RobertaModel, RobertaConfig, AlbertModel, AlbertConfig
import torch

C:\Users\lenovo\.conda\envs\CLMLF\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\lenovo\.conda\envs\CLMLF\lib\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
C:\Users\lenovo\.conda\envs\CLMLF\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [6]:
bert_config = BertConfig.from_pretrained('../../bert/bert-base-uncased/')
model = BertForPreTraining.from_pretrained('../../bert/bert-base-uncased/', config=bert_config)
bert_model = model.bert

Some weights of BertForPreTraining were not initialized from the model checkpoint at ../../bert/bert-base-uncased/ and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
text=['hello boy','i am a fish']
tokenizer = BertTokenizer.from_pretrained('../../bert/bert-base-uncased/vocab.txt')
text_token_list = [tokenizer.tokenize('[CLS]' + t + '[SEP]') for t in text]
text_to_id = [tokenizer.convert_tokens_to_ids(text_token) for text_token in text_token_list]

C:\Users\lenovo\.conda\envs\CLMLF\lib\site-packages\transformers\tokenization_utils_base.py:1643: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  FutureWarning,


In [9]:
text_token_list

[['[CLS]', 'hello', 'boy', '[SEP]'],
 ['[CLS]', 'i', 'am', 'a', 'fish', '[SEP]']]

In [10]:
text_to_id

[[101, 7592, 2879, 102], [101, 1045, 2572, 1037, 3869, 102]]

In [11]:
data_length=[len(i) for i in text_to_id]
text_to_id = [torch.LongTensor(b) for b in text_to_id]
import torch.nn.utils.rnn as run_utils

text_to_id = run_utils.pad_sequence(text_to_id, batch_first=True, padding_value=0)
text_to_id

tensor([[ 101, 7592, 2879,  102,    0,    0],
        [ 101, 1045, 2572, 1037, 3869,  102]])

In [12]:
max_l=text_to_id.shape[1]
att=[]
for length in data_length:
    text_mask_cell = [1] * length
    text_mask_cell.extend([0] * (max_l - length))
    att.append(text_mask_cell[:])
att

[[1, 1, 1, 1, 0, 0], [1, 1, 1, 1, 1, 1]]

In [18]:
att = torch.LongTensor(att)
bert_output=bert_model(text_to_id, attention_mask=att)
bert_output=bert_output.last_hidden_state
bert_output[0]

tensor([[-1.2335e-01,  2.2231e-01,  2.1620e-02,  ..., -4.4586e-02,
          6.5442e-02,  2.2256e-01],
        [-6.3776e-01, -2.3101e-01,  9.7291e-01,  ...,  3.6812e-01,
          1.5373e-01,  4.5564e-02],
        [-3.4176e-01, -5.4559e-01,  4.7940e-01,  ...,  4.7953e-01,
          2.8465e-01, -5.8523e-01],
        [ 7.9941e-01,  1.3761e-02, -1.9604e-01,  ...,  1.8861e-01,
         -5.4835e-01, -1.9504e-01],
        [-5.8901e-01,  7.0094e-02,  7.0958e-01,  ...,  2.0668e-02,
         -9.5173e-04,  5.2641e-02],
        [-5.9858e-01, -2.3409e-01,  3.3245e-01,  ...,  3.7088e-01,
          3.9300e-02, -1.1559e-01]], grad_fn=<SelectBackward>)

可以看到，上述输出的 4、5位置并不是0，所以在后续进行词向量的相关操作时，我们有理由对这部分进行mask

接下来我们就应该思考，什么时候mask需要被考虑，什么时候可以不考虑
当然一种简单的方法是什么时候都考虑

在开始之前，我们先记住相关的维度信息，根据笔者的经验（虽然不多），这些维度信息对相关分析都会有很大的帮助

In [19]:
bert_output.size()

torch.Size([2, 6, 768])

In [20]:
att.size()

torch.Size([2, 6])

In [21]:
text_to_id.size()

torch.Size([2, 6])

# （2）linear

In [23]:

import torch.nn as nn
import torch.nn.functional as FF

In [33]:
att.unsqueeze(-1)

tensor([[[1],
         [1],
         [1],
         [1],
         [0],
         [0]],

        [[1],
         [1],
         [1],
         [1],
         [1],
         [1]]])

In [27]:
linear_model = torch.nn.Linear(768, 1)


In [28]:
linear_output=linear_model(bert_output)

In [29]:
linear_output

tensor([[[ 0.1522],
         [ 0.2799],
         [-0.0567],
         [ 0.3866],
         [-0.0196],
         [-0.1301]],

        [[ 0.2027],
         [ 0.4311],
         [ 0.4463],
         [ 0.5477],
         [-0.0103],
         [ 0.5155]]], grad_fn=<AddBackward0>)

In [54]:
att_mask=att.unsqueeze(1)

In [55]:
att_mask=att_mask.repeat(1,768,1)

In [56]:
att_mask=1-att_mask.permute(0,2,1)

In [57]:
att_mask.byte()

tensor([[[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]], dtype=torch.uint8)

In [65]:
masked_bert_output=bert_output.data.masked_fill(att_mask.byte(),0)

C:\Users\lenovo\.conda\envs\CLMLF\lib\site-packages\ipykernel_launcher.py:1: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at  ..\aten\src\ATen\native\TensorAdvancedIndexing.cpp:650.)
  """Entry point for launching an IPython kernel.


In [66]:
masked_bert_output

tensor([[[-0.1234,  0.2223,  0.0216,  ..., -0.0446,  0.0654,  0.2226],
         [-0.6378, -0.2310,  0.9729,  ...,  0.3681,  0.1537,  0.0456],
         [-0.3418, -0.5456,  0.4794,  ...,  0.4795,  0.2846, -0.5852],
         [ 0.7994,  0.0138, -0.1960,  ...,  0.1886, -0.5484, -0.1950],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[-0.0762,  0.3267, -0.1818,  ..., -0.1450,  0.3118,  0.5097],
         [ 0.2070,  0.3888, -0.1240,  ..., -0.3377,  0.5565,  0.3100],
         [ 0.0976,  0.3917,  0.0323,  ..., -0.4654,  0.4817,  0.4301],
         [ 0.1149,  0.3431,  0.0817,  ..., -0.2893,  0.3845,  0.8167],
         [-0.1979, -0.2843, -0.4806,  ...,  0.2773,  0.6106, -0.5674],
         [ 0.8017,  0.1640, -0.3317,  ..., -0.0054, -0.7692, -0.4068]]])

In [67]:
masked_linear_output=linear_model(masked_bert_output)

In [68]:
masked_linear_output

tensor([[[ 0.1522],
         [ 0.2799],
         [-0.0567],
         [ 0.3866],
         [ 0.0183],
         [ 0.0183]],

        [[ 0.2027],
         [ 0.4311],
         [ 0.4463],
         [ 0.5477],
         [-0.0103],
         [ 0.5155]]], grad_fn=<AddBackward0>)

In [70]:
linear_output

tensor([[[ 0.1522],
         [ 0.2799],
         [-0.0567],
         [ 0.3866],
         [-0.0196],
         [-0.1301]],

        [[ 0.2027],
         [ 0.4311],
         [ 0.4463],
         [ 0.5477],
         [-0.0103],
         [ 0.5155]]], grad_fn=<AddBackward0>)

In [78]:
linear_output.shape

torch.Size([2, 6, 1])

In [69]:
# 到这里我们就可以发现，实际上是不构成任何影响的，从理论的角度来说，这部分在最后loss的时候屏蔽即可

# （3）max/min

In [81]:
# 这边也是同理，虽然会增加一些计算开销，但实际上对
max_output=torch.min(linear_output,dim=2)
max_output

torch.return_types.min(
values=tensor([[ 0.1522,  0.2799, -0.0567,  0.3866, -0.0196, -0.1301],
        [ 0.2027,  0.4311,  0.4463,  0.5477, -0.0103,  0.5155]],
       grad_fn=<MinBackward0>),
indices=tensor([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]]))

In [82]:
max_output=torch.min(linear_output,dim=1)
max_output

torch.return_types.min(
values=tensor([[-0.1301],
        [-0.0103]], grad_fn=<MinBackward0>),
indices=tensor([[5],
        [4]]))

In [ ]:
# 这里就出现问题了，所以在max和min的时候要注意遮掩，具体看使用max还是使用min

# （4） RNN

In [83]:
lstm = torch.nn.LSTM(input_size=768,hidden_size=50,num_layers=1, bidirectional=True)

In [84]:
# 这里先介绍两个函数
# pack_padded_sequence 和 pad_packed_sequence

pack_padded_sequence
这个函数主要做了两件事： pad 和封装
因为在rnn模型中，一般先将batch中的数据按照一个时间步一个时间步喂入模型的，这个包的主要作用就是将按照样本堆叠的数据，抽取出时间步这个维度重新堆叠。

input： pad_sequence 的结果
length：batch 中各个句子的实际长度
batch_first: batch 是否在第一位，默认值是False，上面的例子指定为了True，因为是二维，方便观察理解，一般放入lstm或者gru是需要时间步放在第一位的。
enforce_sorted：如果是 True ，则输入应该是按长度降序排序的序列。如果是 False ，会在函数内部进行排序。默认值为 True。
需要注意的是，默认条件下，我们必须把输入数据按照序列长度从大到小排列后才能送入 pack_padded_sequence ，否则会报错。

In [ ]:
#注意上面所说，需要把输入数据按照序列长度从大到小排列后才能送入